# Visual Odometry (VO)

In this assignment, you do not need. a GPU. You will use the pykitti module and KITTI odometry dataset.

You can download the odometry data from [here](https://drive.google.com/file/d/1PJOUnM3nEwDpqiRvfqUnkNPJZpM4PKYV/view?usp=sharing).

## Monocular VO with OpenCV on KITTI

For each consecutive frame pair in the sequence, you will compute the relative pose between the frames and visualize it. You will use:

* pykitti code similar to what you wrote in mvs part to load the seqeunce with ground-truth info. (Check out the [demo code](https://github.com/utiasSTARS/pykitti/blob/master/demos/demo_odometry.py))
* OpenCV functions to compute and visualize the features and the essential matrix.

Please follow these steps to complete the assignment:

1. You can use the ORB Feature to do the feature matching:
    `orb = cv2.ORB_create()` to create the ORB object
    and then `orb.detectAndCompute()` to find the keypoints and descriptors on both frames

2. You can use brute-force matcher to match ORB descriptors:
    `bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)`

3. After matching the descriptors, sort the matched keypoints.

4. Draw matches on the two images using the `cv2.drawMatches()` function.

5. Compute the essential matrix using the `cv2.findEssentialMat()` function. Note that you need the matching points and the instrinsics for this function. 

6. Extract the rotation and translation from the essential matrix using the `cv2.recoverPose()` function.

7. Multiply the estimated rotation and translation with the previous rotation and translation. Initialize rotation to identity and translation to zeros on the first frame.

8. Display the current image with the keypoints on it using the `cv2.drawKeypoints()` function.

9. Update the previous rotation and translation as the current rotation and translation.

10. Draw the estimated trajectory as blue and ground-truth trajectory as green. You can use the `cv2.circle()` function.


You can create a video of your visualization of images and poses for the provided sequence.

**Bonus**: Compute the absolute trajectory error between the estimated trajectory and the ground-truth trajectory. 

Some examples repositories that might be useful:
* https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/
* https://github.com/uoip/monoVO-python


In [31]:
######## DISCLAIMER: I have borrowed most of the code from the given example repository: https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/
# I have read and understood the given code, I just thought it would be meaningless to try and change the variable names etc., so most of the code is the same with this repo

import os
import imageio
import numpy as np
import cv2
from glob import glob
import pykitti

basedir = "KITTI_odometry/"
sequence = "09"
# Using the images from camera 2
img_data_dir = "KITTI_odometry/sequences/" + sequence + '/image_2/'
dataset = pykitti.odometry(basedir, sequence)

# MOST OF THE CODE BELOW IS BORROWED FROM: https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/vo.py

len_trajMap = 370 # I tried to set this to 1000, which was the default value in the given repository, but I encountered an error saying that this should be 370
trajMap = np.zeros((len_trajMap, len_trajMap, 3), dtype=np.uint8)

# get the camera 2 intrinsic parameters
calibration = dataset.calib.K_cam2
fx = calibration[0,0]
fy = calibration[1,1]
cx = calibration[0,2]
cy = calibration[1,2]

# image directory
img_list = glob(img_data_dir + '/*.png')
img_list.sort()
num_frames = len(img_list)

# these lists are for creating the videos
matched_images = []
keypoints = []
trajectories = []

# to keep the errors
errors = []
for i in range(num_frames):
    curr_img = cv2.imread(img_list[i], 0)
    if i == 0:
        # initializing rotation to identity and translation to zeros for the first frame
        curr_R = np.eye(3)
        curr_t = np.array([0, 0, 0])
    else:
        prev_img = cv2.imread(img_list[i-1], 0)

        # 1. orb features
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(prev_img, None)
        kp2, des2 = orb.detectAndCompute(curr_img, None)

        # 2. match descriptors with the brute force matcher
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)

        # 3. sorting the matches
        matches = sorted(matches, key=lambda x: x.distance)

        # 4. drawing the matches
        img_matching = cv2.drawMatches(prev_img, kp1, curr_img, kp2, matches[0:100], None)
        cv2.imwrite(f'matched_images/{i}.png', img_matching)
        matched_images.append(imageio.imread(f'matched_images/{i}.png'))
        
        # 5. compute essential matrix
        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
        E, mask = cv2.findEssentialMat(pts1, pts2, focal=fx, pp=(cx, cy), method=cv2.RANSAC, prob=0.999, threshold=1)
        pts1 = pts1[mask.ravel() == 1]
        pts2 = pts2[mask.ravel() == 1]
        
        # 6. extract the rotation and the translation
        _, R, t, mask = cv2.recoverPose(E, pts1, pts2, focal=fx, pp=(cx, cy))
        
        # 7. multiplying with the previous rotation and translation to obtain the camera motion
        R = R.transpose()
        t = -np.matmul(R, t)

        if i == 1:
            curr_R = R
            curr_t = t
        else:
            curr_R = np.matmul(prev_R, R)
            curr_t = np.matmul(prev_R, t) + prev_t
            
        # 8. draw the current image with keypoints
        curr_img_kp = cv2.drawKeypoints(curr_img, kp2, None, color=(0, 255, 0), flags=0)
        cv2.imwrite(f'keypoints/{i}.png', curr_img_kp)
        keypoints.append(imageio.imread(f'keypoints/{i}.png'))
        
        # 9. update the previous rotation and translation
        prev_R = curr_R
        prev_t = curr_t
        
        # obtain the ground truth trajectory
        gt = dataset.poses[i][0:3,-1]
        
        # calculate and save the absolute trajectory error (mean for all coordinates)
        # to calculate the error, i simply take the difference between the predicted trajectory and the 
        # ground truth trajectory, take the absolute value and take mean across all the coordinates
        errors.append(np.mean(np.abs(gt - curr_t.squeeze())))

        # 10. drawing the estimated and gt trajectories
        offset_draw = (int(len_trajMap/2))
        cv2.circle(trajMap, (int(gt[0])+offset_draw, int(gt[2])+offset_draw), 1, (0,255,0), 2) # green for the ground truth
        cv2.circle(trajMap, (int(curr_t[0])+offset_draw, int(curr_t[2])+offset_draw), 1, (255,0,0), 2) # blue for the estimation
        
        cv2.imwrite(f'trajectories/{i}.png', trajMap.astype(np.uint8))
        trajectories.append(imageio.imread(f'trajectories/{i}.png'))

# calculate the mean error
absolute_trajectory_error = np.mean(errors)
print(f'Mean absolute trajectory error: {absolute_trajectory_error}')

# save the frames as a gif file
print('Creating videos')
imageio.mimsave("matched_images.gif", matched_images)
imageio.mimsave("keypoints.gif", keypoints)
imageio.mimsave("trajectories.gif", trajectories)
print('Done')

Mean absolute trajectory error: 4.606227611036428
Creating videos
Done
